In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow import keras
from keras import Sequential,layers
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout,AveragePooling2D,Softmax
from tensorflow.keras.layers import ReLU,GlobalAveragePooling3D
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.losses import BinaryCrossentropy
import numpy as np

2024-06-27 12:10:50.631869: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-27 12:10:50.632013: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-27 12:10:50.779420: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
train_dir = '/kaggle/input/dogs-vs-cats/train'
test_dir = '/kaggle/input/dogs-vs-cats/test'

In [5]:
train_dataset = image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='int',
    shuffle=True
)
test_dataset = image_dataset_from_directory(
    test_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='int',
    shuffle=False
)


Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [6]:
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label
train_dataset = train_dataset.map(preprocess)
test_dataset = test_dataset.map(preprocess)



In [5]:
train_images = []
train_labels = []
for images, labels in train_dataset:
    train_images.append(images)
    train_labels.append(labels)

test_images = []
test_labels = []
for images, labels in test_dataset:
    test_images.append(images)
    test_labels.append(labels)

# Convert to tensors
train_images = tf.concat(train_images, axis=0)
train_labels = tf.concat(train_labels, axis=0)
test_images = tf.concat(test_images, axis=0)
test_labels = tf.concat(test_labels, axis=0)

# Check shapes
print(f"Train images shape: {train_images.shape}")
print(f"Train labels shape: {train_labels.shape}")
print(f"Test images shape: {test_images.shape}")
print(f"Test labels shape: {test_labels.shape}")

ResourceExhaustedError: {{function_node __wrapped__ConcatV2_N_625_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[20000,224,224,3] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:ConcatV2] name: concat

In [7]:
class inception(keras.Model):
    def __init__(self,filter_1,filter_3,filter_5):
        super(inception,self).__init__()
        self.conv1=Conv2D(filter_1,kernel_size=(1,1),strides=1,padding='same',activation='relu')
        self.seq3=Sequential([Conv2D(filter_1,kernel_size=(1,1),strides=1,padding='same',activation='relu'),
                              Conv2D(filter_3,kernel_size=(3,3),strides=1,padding='same',activation='relu')
                              ])
        self.seq5=Sequential([Conv2D(filter_1,kernel_size=(1,1),strides=1,padding='same',activation='relu'),
                              Conv2D(filter_5,kernel_size=(5,5),strides=1,padding='same',activation='relu')
                             ])
        self.seq=Sequential([MaxPooling2D(pool_size=(3,3),strides=1,padding='same'),
                             Conv2D(filter_1,kernel_size=(1,1),strides=1,padding='same',activation='relu')
                             ])
    def call(self,inputs):
        x1=self.conv1(inputs)
        x2=self.seq3(inputs)
        x3=self.seq5(inputs)
        x4=self.seq(inputs)
        return tf.concat([x1,x2,x3,x4],axis=-1)


In [8]:
class Network(keras.Model):
  def __init__(self):
    super(Network,self).__init__()
    self.seq1=Sequential([Conv2D(64,kernel_size=(7,7),strides=2,padding='same',activation='relu'),
                          MaxPooling2D(pool_size=(3,3),strides=2,padding='valid'),
                          BatchNormalization(),
                          Conv2D(64,kernel_size=(1,1),strides=1,padding='valid',activation='relu'),
                          Conv2D(192,kernel_size=(3,3),strides=1,padding='same',activation='relu'),
                          BatchNormalization(),
                          MaxPooling2D(pool_size=(3,3),strides=2,padding='same'),
                          inception(64,96,32),
                          inception(128,128,96),
                          MaxPooling2D(pool_size=(3,3),strides=2,padding='same'),
                          inception(192,96,32)
                               ])
    self.classify1=Sequential([AveragePooling2D(pool_size=(5,5),strides=3,padding='valid'),
                               Conv2D(160,kernel_size=(1,1),strides=1,padding='same',activation='relu'),
                               Flatten(),
                               Dense(1024,activation='relu'),
                               Dropout(0.7),
                               Dense(512,activation='relu'),
                               Dropout(0.7),
                               Dense(1,activation='sigmoid')
                               ])
    self.seq2=Sequential([inception(160,112,80),
                          inception(128,128,128),
                          inception(112,144,160)
                              ])
    self.classify2=Sequential([AveragePooling2D(pool_size=(5,5),strides=3,padding='valid'),
                               Conv2D(112,kernel_size=(1,1),strides=1,padding='same',activation='relu'),
                               Flatten(),
                               Dense(1024,activation='relu'),
                               Dropout(0.7),
                               Dense(512,activation='relu'),
                               Dropout(0.7),
                               Dense(1,activation='sigmoid')
                               ])
    self.seq3=Sequential([inception(256,160,160),
                          MaxPooling2D(pool_size=(3,3),strides=2,padding='same'),
                          inception(256,160,160),
                          inception(384,192,64),
                          AveragePooling2D(pool_size=(7,7),strides=1,padding='valid'),
                          Flatten(),
                          Dropout(0.4),
                          Dense(512,activation='relu'),
                          Dropout(0.4),
                          Dense(256,activation='relu'),
                          Dropout(0.4),
                          Dense(1,activation='sigmoid')
                          ])

  def call(self,inputs):
    x=self.seq1(inputs)
    classifier1=tf.squeeze(self.classify1(x))
    x=self.seq2(x)
    classifier2=tf.squeeze(self.classify2(x))
    final_classifier=tf.squeeze(self.seq3(x))

    
    return classifier1,classifier2,final_classifier

In [ ]:
class Inception(Model):
    def __init__(self, filter_1, filter_3, filter_5):
        super(Inception, self).__init__()
        self.conv1 = Conv2D(filter_1, kernel_size=(1, 1), strides=1, padding='same', activation='relu')
        self.seq3 = Sequential([
            Conv2D(filter_1, kernel_size=(1, 1), strides=1, padding='same', activation='relu'),
            Conv2D(filter_3, kernel_size=(3, 3), strides=1, padding='same', activation='relu')
        ])
        self.seq5 = Sequential([
            Conv2D(filter_1, kernel_size=(1, 1), strides=1, padding='same', activation='relu'),
            Conv2D(filter_5, kernel_size=(5, 5), strides=1, padding='same', activation='relu')
        ])
        self.seq = Sequential([
            MaxPooling2D(pool_size=(3, 3), strides=1, padding='same'),
            Conv2D(filter_1, kernel_size=(1, 1), strides=1, padding='same', activation='relu')
        ])

    def call(self, inputs):
        x1 = self.conv1(inputs)
        x2 = self.seq3(inputs)
        x3 = self.seq5(inputs)
        x4 = self.seq(inputs)
        return tf.concat([x1, x2, x3, x4], axis=-1)


class TransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = Sequential([
            Dense(ff_dim, activation='relu'),
            Dense(embed_dim)
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


class Network(Model):
    def __init__(self):
        super(Network, self).__init__()
        self.seq1 = Sequential([
            Conv2D(64, kernel_size=(7, 7), strides=2, padding='same', activation='relu'),
            MaxPooling2D(pool_size=(3, 3), strides=2, padding='valid'),
            BatchNormalization(),
            Conv2D(64, kernel_size=(1, 1), strides=1, padding='valid', activation='relu'),
            Conv2D(192, kernel_size=(3, 3), strides=1, padding='same', activation='relu'),
            BatchNormalization(),
            MaxPooling2D(pool_size=(3, 3), strides=2, padding='same'),
            Inception(64, 96, 32),
            Inception(128, 128, 96),
            MaxPooling2D(pool_size=(3, 3), strides=2, padding='same'),
            Inception(192, 96, 32)
        ])
        
        self.transformer = TransformerBlock(embed_dim=384, num_heads=4, ff_dim=512)

        self.classify1 = Sequential([
            AveragePooling2D(pool_size=(5, 5), strides=3, padding='valid'),
            Conv2D(160, kernel_size=(1, 1), strides=1, padding='same', activation='relu'),
            Flatten(),
            Dense(1024, activation='relu'),
            Dropout(0.7),
            Dense(512, activation='relu'),
            Dropout(0.7),
            Dense(1, activation='sigmoid')
        ])
        self.seq2 = Sequential([
            Inception(160, 112, 80),
            Inception(128, 128, 128),
            Inception(112, 144, 160)
        ])
        self.classify2 = Sequential([
            AveragePooling2D(pool_size=(5, 5), strides=3, padding='valid'),
            Conv2D(112, kernel_size=(1, 1), strides=1, padding='same', activation='relu'),
            Flatten(),
            Dense(1024, activation='relu'),
            Dropout(0.7),
            Dense(512, activation='relu'),
            Dropout(0.7),
            Dense(1, activation='sigmoid')
        ])
        self.seq3 = Sequential([
            Inception(256, 160, 160),
            MaxPooling2D(pool_size=(3, 3), strides=2, padding='same'),
            Inception(256, 160, 160),
            Inception(384, 192, 64),
            AveragePooling2D(pool_size=(7, 7), strides=1, padding='valid'),
            Flatten(),
            Dropout(0.4),
            Dense(512, activation='relu'),
            Dropout(0.4),
            Dense(256, activation='relu'),
            Dropout(0.4),
            Dense(1, activation='sigmoid')
        ])

    def call(self, inputs):
        x = self.seq1(inputs)
        x = self.transformer(x, training=True)
        classifier1 = tf.squeeze(self.classify1(x))
        x = self.seq2(x)
        classifier2 = tf.squeeze(self.classify2(x))
        final_classifier = tf.squeeze(self.seq3(x))
        return classifier1, classifier2, final_classifier


In [33]:
initial_weights = model.get_weights()

In [36]:
initial_weights[0].shape

(7, 7, 3, 64)

In [9]:
model=Network()
optimizer=Adam(learning_rate=0.001)
loss_fn=BinaryCrossentropy()
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        classifier1,classifier2,final_classifier= model(images)
        classifier1_loss = loss_fn(labels, classifier1)
        classifier2_loss = loss_fn(labels, classifier2)
        final_classifier_loss = loss_fn(labels, final_classifier)
        final_loss = classifier1_loss + classifier2_loss + final_classifier_loss
    gradients = tape.gradient(final_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return final_loss,final_classifier


In [10]:
def calculate_accuracy(labels, predictions):
    predictions = tf.sigmoid(predictions)  
    predicted_classes = tf.round(predictions)  
    labels = tf.squeeze(tf.cast(labels, tf.int32))  
    predicted_classes = tf.squeeze(tf.cast(predicted_classes, tf.int32))  
    accuracy = tf.reduce_mean(tf.cast(tf.equal(labels, predicted_classes), tf.float32))
    return accuracy

def calculate_precision(labels, predictions):
    predictions = tf.sigmoid(predictions)  
    predicted_classes = tf.round(predictions)  
    labels = tf.squeeze(tf.cast(labels, tf.int32))  
    predicted_classes = tf.squeeze(tf.cast(predicted_classes, tf.int32))  
    
    true_positives = tf.reduce_sum(tf.cast(predicted_classes * labels, tf.float32))
    predicted_positives = tf.reduce_sum(tf.cast(predicted_classes, tf.float32))
    
    precision = true_positives / (predicted_positives + tf.keras.backend.epsilon())
    return precision


In [12]:
epochs = 10
for epoch in range(epochs):
    total_loss = 0.0
    total_accuracy = 0.0
    total_precision = 0.0
    num_batches = 0
    initial_weights = model.get_weights()
    for images, labels in train_dataset:
        loss, predictions = train_step(images, labels)
        batch_accuracy = calculate_accuracy(labels, predictions)
        batch_precision = calculate_precision(labels, predictions)

        total_loss += loss
        total_accuracy += batch_accuracy
        total_precision += batch_precision
        num_batches += 1
        
    
    average_loss = total_loss / num_batches
    average_accuracy = total_accuracy / num_batches
    average_precision = total_precision / num_batches
    updated_weights = model.get_weights()
    total_count=0
    count=0
    for initial, updated in zip(initial_weights, updated_weights):
        total_count+=1
        if not np.array_equal(initial, updated):
            count+=1
    print(f'Total Count : {total_count} ; Count :{count}')
    print(f"Epoch {epoch + 1}, Loss: {average_loss.numpy()}, Accuracy: {average_accuracy.numpy()}, Precision: {average_precision.numpy()}")

Total Count : 144 ; Count :129
Epoch 1, Loss: 2.0795719623565674, Accuracy: 0.5, Precision: 0.5
Total Count : 144 ; Count :126
Epoch 2, Loss: 2.0795915126800537, Accuracy: 0.5, Precision: 0.5
Total Count : 144 ; Count :140
Epoch 3, Loss: 2.082878828048706, Accuracy: 0.5, Precision: 0.5
Total Count : 144 ; Count :115
Epoch 4, Loss: 2.079641342163086, Accuracy: 0.5, Precision: 0.5
Total Count : 144 ; Count :114
Epoch 5, Loss: 2.0796010494232178, Accuracy: 0.5, Precision: 0.5
Total Count : 144 ; Count :110
Epoch 6, Loss: 2.0795884132385254, Accuracy: 0.5, Precision: 0.5
Total Count : 144 ; Count :109
Epoch 7, Loss: 2.079555034637451, Accuracy: 0.5, Precision: 0.5
Total Count : 144 ; Count :112
Epoch 8, Loss: 2.0795764923095703, Accuracy: 0.5, Precision: 0.5
Total Count : 144 ; Count :116
Epoch 9, Loss: 2.0795483589172363, Accuracy: 0.5, Precision: 0.5
Total Count : 144 ; Count :117
Epoch 10, Loss: 2.0795915126800537, Accuracy: 0.5, Precision: 0.5


In [32]:
initial_weights = model.get_weights()